# Apuntes Aprendizaje Reforzado

Fuente: https://www.youtube.com/watch?v=Mut_u40Sqz4

In [1]:
#!pip install stable-baselines3[extra]
#!pip install gym

In [2]:
import os
import gym
from stable_baselines3 import PPO
#Esto es el el algoritmo de aprendizaje que vamos a utilizar, aunque hay muchos diferentes
# y puede que este no sea el que nosotros necesitemos
from stable_baselines3.common.vec_env import DummyVecEnv
# DummyVecEnv nos sirve para vectorizar nuestros entornos y es maravilloso porque nos
# posibilita hacer muchas simulaciones en un vector, lo que acelera el proceso de manera extrema
from stable_baselines3.common.evaluation import evaluate_policy
# Esta libraria es para evaluar modelos

## Cargar Entorno

In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)
# Vamos a trabajar con un problema de RL que consiste en mantener una barra vertical sin que caiga
# La barra se encuentra sobre una plataforma móvil
# El problema con el que trabajamos y cómo este se describe es lo que llamamos el entorno

C:\Users\jpeir\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [4]:
episodes = 5
for ep in range(episodes):
    state = env.reset()
    # Reinicia el entorno
    done = False
    # Estado del entorno
    score = 0
    # Recompensa/Puntuación
    
    while not done:
        env.render()
        # Muestra una interfaz del problema
        action = env.action_space.sample()
        # Realiza una acción aleatoria
        # En este problema, como tal y como se define en el entorno, solo se pueden hacer 2 posibles opciones, izquierda o derecha
        # Así que action_space es el espacio de posibles acciones, que como además son acciones discretas el objeto se guarda como Discrete(2)
        # Luego, cuando hacemos sample tomamos una de las 2 posibles acciones aleatoriamente
        n_state, reward, done, trunc, info = env.step(action)
        # Ejecutamos la accion elegida con step
        # Devuelve el estado al que pasamos (la definición del estado)
        # La recompensa que adquiere en ese paso
        # Si la mision se ha completado
        # Si la mision ha podido completarse en menos del maximo de pasos posibles, (muchas veces no lo devuelve)
        # Informacion adicional
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:57.0
Episode:1 Score:13.0
Episode:2 Score:10.0
Episode:3 Score:29.0
Episode:4 Score:16.0


C:\Users\jpeir\anaconda3\lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
C:\Users\jpeir\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
 env.reset()
env.action_space.sample()
env.step(action)

(array([ 0.03707751,  0.15331598, -0.04880498, -0.3061951 ], dtype=float32),
 1.0,
 False,
 False,
 {})

## Entrenar modelo

In [6]:
log_path = os.path.join("Entrenamiento", "Logs")
# Es muy importante que los directorios estén creados antes de hacer nada
# Aquí hemos creado una carpeta llamada Entrenamiento, dentro la cual hay una llamada Logs para guardar progreso
# Y otra Saved para guardar modelos entrenados
log_path

'Entrenamiento\\Logs'

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# Con esta linea creamos un entorno vectorizado, lo que nos permite paralelizar nuestro entorno y hacer muchos entrenamientos en paralelo
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)
# Aquí estamos creando el modelo con el algoritmo que hemos importado desde el principio y que decidimos que seria el que utilizaremos
# Hay muchos más algoritmos, pero para este ejemplo hemos preferido usar este
# El primer parametro es la política que utilizaremos, MlpPolicy viene de Multi-Layer Perceptron Policy
# Hay 3 posibles politicas de stable_baselines3: MlpPolicy, CnnPolicy, MultiInputPolicy
# Segundo atributo el entorno, (que ya hemos vectorizaodo antes)
# Luego verbose, porque queremos ver lo que ocurre, y la ruta de guardado, creada antes

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [8]:
#PPO??
# Hay chorrocientos atributos más

In [9]:
model.learn(total_timesteps = 20000)
# Y voilá, así se entrena un modelín, no es dificil
# EL resultado del entrenamiento se guarda en una carpeta
# Cada vez que se ejecuta el bicho se crea una carpeta con lo que ha ido aprendiendo

Logging to Entrenamiento\Logs\PPO_3


C:\Users\jpeir\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 1252 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1004         |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0076799016 |
|    clip_fraction        | 0.0952       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | -0.00998     |
|    learning_rate        | 0.0003       |
|    loss                 | 5.21         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0147      |
|    value_loss           | 45.9         |
------------------------------------------
----------------

## Guardar y recargar un modelo

In [10]:
PPO_Path = os.path.join("Entrenamiento", "Saved Models", "PPO_CartPole")
# Creamos una carpeta para guardar modelos y reejecutarlos, no hace falta que ya exista, sino que la creará
# La creará como un zip
model.save(PPO_Path)

In [11]:
# Si quisieramos, ahora ya tenemos el modelo guardado así que podemos cargarlo de nuevo
# Podemos eliminar el modelo que tenemos en memoria
del model # Eliminar modelo
# model.learn(total_timesteps = 20000) # Esto daría error porque la variable ya no existe

In [12]:
# Y lo podemos importar de la carpeta que teniamos guardada
model = PPO.load(PPO_Path, env = env)

## Evaluar modelo

In [13]:
# Usamos la función que hemos importado al principio para evaluar modelos
evaluate_policy(model, env, n_eval_episodes = 10, render=True)
# Podemos elegir el número de episodios a evaluar
# Y con render elegimos si queremos verlo ejecutar o no

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(200.0, 0.0)

In [14]:
env.close() # Y cerramos

## Testear

In [15]:
# Ya hemos evaluado el modelo, pero ahora, la pregunta realmente importante es, qué ocurrirá si cambia el entorno
# Es el motivo por el que hacemos todo esto
# Tomamos la idea del código del principio

episodes = 5 # Numero episodios
for ep in range(episodes):
    obs = env.reset()
    # Reiniciamos el entorno, para ver qué pasa si cambia
    # Guardamos el entorno el una variable
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        # Esto es casi lo único que cambia respecto al principio
        # Ahora en lugar de acciones aleatorias va a predecir las acciones
        # Tenemos que pasarle el entorno para que prediga
        obs, reward, done, info = env.step(action)
        # Y el entorno se tiene que actualizar en cada iteracion
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:[200.]
Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]


In [16]:
env.close()

## Ver registros de entrenamiento

In [17]:
training_log_path = os.path.join(log_path, "PPO_2") # Ruta al entrenamiento que queremos ver
training_log_path

'Entrenamiento\\Logs\\PPO_2'

In [20]:
# Meter un comando iniciado con ! en jupyter permite hacer comandos para interactuar con un terminal
#!tensorboard --logdir={training_log_path}
# Cuando ejecutas esto, es muy guay, te permite ver los registros que conforma el entrenamiento del modelo
# Esto debería devolver un link a localhost, si lo buscas en un busacdor te ofrece mucha información acerca del entrenamiento

## Callback

In [22]:
# Esto servirá para parar el entrenamiento de un modelo cuando se ha alcanzado una puntuación determinada
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
# Dejar de entrenar en umbral de recompensa, para evitar modelo gigantes e inestables

In [23]:
save_path = os.path.join("Entrenamiento", "Saved Models")
# Tendremso que especificar la carpeta para guardar el modelo

stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose = 1)
# Ponemos que acabe de entrenar cuando consiga una recompensa de 200
# La palabra threshold significa umbral

eval_callback = EvalCallback(env, callback_on_new_best=stop_callback,
                             eval_freq=10000, best_model_save_path = save_path,
                             verbose = 1)
# Ahora, hacemos una función que evalue la parada, la callback
# Le pasamos el entorno, la callback que hemos creado
# Podemos especificar cuanto de frecuente queremos que se evalue esta callback
# Esto hace que después de cada 10000 iteraciones empiece a comprobar que se está sobrepasando la recompensa
# Luego, si encuentra que se ha mejorado el modelo, para y se le indica el path donde debe guardar el modelo

In [24]:
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)
# Ahora creamos un modelo, exacatemnte igual que hemos creado al principio
# Lo que cambiará es que cuando lo entrenemos le pasaremos la callback
model.learn(total_timesteps = 20000, callback=eval_callback)
# Ahora además del numero de iteraciones vamos a pasarle la callback
# Para que cada 10000 compuebe si hace falta parar
# Podemos ver que al final sale el mensaje "Stopping training because the mean reward 200.00  is above the threshold 200"
# Se cumple la callback
# Y podemos ver que el la carpeta Saved Models aparece una nueva carpeta llamada best_model.zip
# Que es la que ha creado la callback

Using cpu device
Logging to Entrenamiento\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 1230 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 909         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008481778 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00144    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.18        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 50.7        |
-----------------------

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=192.60 +/- 14.80
Episode length: 192.60 +/- 14.80
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 193         |
|    mean_reward          | 193         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008003194 |
|    clip_fraction        | 0.0527      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.611      |
|    explained_variance   | 0.213       |
|    learning_rate        | 0.0003      |
|    loss                 | 32.5        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 67.1        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 752   |
|    iterations      | 5     |
|    ti

## Cambiar politicas

In [26]:
# Ahora veamos cómo cambiariamos la política dentro estructura de la red neuronal de nuestro algoritmo
# Esto no es muy necesario hacerlo a no que haya una intención especifica
net_arch = [dict(pi = [128,128,128,128], vf = [128,128,128,128])]
# Vamos a especificar que tenemos una nueva red neuronal
# Con pi especificamos que tenemos 128 unidades en cada una de las capas
# Especificamos lo mismo para la función valor, en vf

In [27]:
# El procedimiento es exactamente igual que antes
# Pero lo unico que cambia es que al argumento policy_kwargs le pasamos la nueva politica
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path, policy_kwargs={"net_arch":net_arch})
# Vemos que lo pasamos como policy_kwargs={"net_arch":net_arch}
model.learn(total_timesteps = 20000, callback=eval_callback)

Using cpu device
Logging to Entrenamiento\Logs\PPO_5


C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1152 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 650        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01430518 |
|    clip_fraction        | 0.198      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.682     |
|    explained_variance   | -0.0164    |
|    learning_rate        | 0.0003     |
|    loss                 | 2.56       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0199    |
|    value_loss           | 19.6       |
----------------------------------------
-----------------------------------------
| time/   

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.011292972 |
|    clip_fraction        | 0.0858      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.582      |
|    explained_variance   | 0.456       |
|    learning_rate        | 0.0003      |
|    loss                 | 8.39        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0117     |
|    value_loss           | 38.1        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 514   |
|    iterations      | 5     |
|    time_elapsed    | 19    |


## Otros algoritmos

In [28]:
from stable_baselines3 import DQN
# Igual que hemos utilizado el algoritmo PPO hay muchos más que en determinados casos pueden sernos útiles
# Vamos a probar con otro, DQN
# Veremos que es exactamente igual, solo cambiando la función
model = DQN("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)
model.learn(total_timesteps = 20000)
# Vemos que es lo mismo, e igual que antes hemos utilizado PPO.load, se utiliza DQN.load

Using cpu device
Logging to Entrenamiento\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.972    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3741     |
|    time_elapsed     | 0        |
|    total_timesteps  | 58       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.931    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4755     |
|    time_elapsed     | 0        |
|    total_timesteps  | 146      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.887    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4838     |
|    time_elapsed     | 0        |
|    total_timesteps  | 238      |
----------------------------------
--